In [1]:
# 为这个项目导入需要的库
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
# 允许为DataFrame使用display()
from IPython.display import display 
from matplotlib import pyplot as plt
import networkx as nx


## 个人数据（已清洗）

In [2]:
#清洗好的individual data
data_financial = pd.read_csv('data_cleaned2.csv',header=0,error_bad_lines=False)

data_user=pd.read_csv('extraData/NFA_feature_out.csv')
#将inf数据设置为nan
data_user[np.isinf(data_user)] = np.nan
#设置index
data_financial = data_financial.set_index("id_index")
data_user=data_user.set_index("id")
#数据合并
data1=pd.merge(data_user,data_financial,left_index=True,right_index=True)
#删除nan数据
data1=data1.dropna(axis=0,how='any')
#individual data前5条
print('前5条:')
pd.set_option('display.max_columns', None)
display(data_financial.head(5))
print("========")


#数据规模
print('数据规模：{}'.format(data_financial.shape))
print("")
print("========")

#标签比例
print('标签比例：')
display(pd.value_counts(data1['target_tov']))
print("========")

#有空值的列——无
print('有空值的列:无')
print(data1.columns[data1.isna().any()])
print("========")


前5条:


,class,date_month,target_tov,acc_cnt,cre_type,ucreate_days,user_type,wx_firdate_days,wx_regtime_days,wxu_sex,if_bindmobile_safe_max,wxu_bindqq_safe,pay_crd_num,birth_date,sex,if_bindmobile_safe_min,if_chg_mob_max,if_chg_mob_min,if_chg_qq_max,if_chg_qq_min,if_chg_dev_max,if_chg_dev_min,chg_dev_cnt_max,chg_dev_cnt_min,if_op_safdev_max,if_op_safdev_min,unbind_max,unbind_min,old_acc_sus_max,old_acc_sus_min,maochi_lvl_max,maochi_lvl_min,dev_acc_cnt_max,dev_acc_cnt_min,acc_dev_cnt_max,acc_dev_cnt_min,ios_max,ios_min,block_max,block_min,trust_lvl_min,stol_lvl_max,feed_lvl_max,feed_lvl_min,anping_feed_max,anping_feed_min,sms_ttl_cnt,sms_cnt_max,sms_cnt_min,block_ttl_cnt,block_cnt_max,hb_ttl_amt,hb_ttl_cnt,hb_high_40_cnt,hb_low_3_cnt,hb_avg_low_1_cnt,hb_avg_high_100_cnt,hb_max_amt,hb_fpay_ttl_amt,hb_fpay_ttl_cnt,hb_bpay_ttl_amt,hb_bpay_ttl_cnt,hb_am_ttl_cnt,hb_pm_ttl_cnt,hb_ng_ttl_cnt,hb_em_ttl_cnt,tf_ttl_amt,tf_ttl_cnt,tf_max_amt,tf_min_amt,tf_usr_cnt,tf_am_ttl_cnt,tf_pm_ttl_cnt,tf_ng_ttl_cnt,tf_em_ttl_cnt,tf_rev_ttl_amt,tf_rev_ttl_cnt,tf_rev_max_amt,tf_rev_min_amt,tf_am_rev_ttl_cnt,tf_pm_rev_ttl_cnt,tf_ng_rev_ttl_cnt,tf_em_rev_ttl_cnt,tf_fpay_ttl_amt,tf_bpay_ttl_amt,tf_fpay_ttl_cnt,tf_bpay_ttl_cnt,lct_ttl_amt,lct_ttl_cnt,lct_max_amt,lct_prod_cnt,lct_fpay_ttl_amt,lct_fpay_ttl_cnt,lct_bpay_ttl_amt,lct_bpay_ttl_cnt,lct_am_ttl_cnt,lct_pm_ttl_cnt,lct_ng_ttl_cnt,lct_em_ttl_cnt,vir_ttl_amt,vir_ttl_cnt,vir_max_amt,vir_fpay_amt,vir_fpay_cnt,vir_bpay_amt,vir_bpay_cnt,vir_am_ttl_cnt,vir_pm_ttl_cnt,vir_ng_ttl_cnt,vir_em_ttl_cnt,traffic_ticket_cnt,phone_rate_cnt,qq_service_cnt,commonweal_cnt,commonweal_amt,safe_pay_amt,cre_crd_amt,cre_crd_cnt,self_wxcc_amt,other_wxcc_amt,self_wxcc_cnt,other_wxcc_cnt,ttl_amt,ttl_cnt,fpay_ttl_amt,fpay_ttl_cnt,bpay_ttl_amt,bpay_ttl_cnt,am_ttl_cnt,pm_ttl_cnt,ng_ttl_cnt,em_ttl_cnt,region_3_unify_cnt,tf_rev_usr_cnt,welfare_amt,welfare_cnt,welfare_max_amt,tencent_amt,tencent_cnt,tencent_max_amt,ecc_amt,ecc_cnt,ecc_max_amt,living_amt,living_cnt,living_max_amt,game_pay_amt,game_pay_cnt,game_pay_max_amt,game_pay_min_amt,web_vir_amt,web_vir_cnt,web_vir_max_amt,mob_rechg_amt,mob_rechg_cnt,mob_rechg_max_amt,trav_hotel_amt,trav_hotel_cnt,trav_hotel_max_amt,life_sev_amt,life_sev_cnt,life_sev_max_amt,mob_rechg_3th_amt,mob_rechg_3th_cnt,mob_rechg_3th_max_amt,mall_market_amt,mall_market_cnt,mall_market_max_amt,logis_amt,logis_cnt,logis_max_amt,beauty_amt,beauty_cnt,beauty_max_amt,car_amt,car_cnt,car_max_amt,baby_amt,baby_cnt,baby_max_amt,o2o_amt,o2o_cnt,o2o_max_amt,video_amt,video_cnt,video_max_amt,film_amt,film_cnt,film_max_amt,travel_amt,travel_cnt,travel_max_amt,tf_resid_cnt,tf_resid_amt,tf_card_cnt,tf_card_amt,tf_strange_amt,tf_strange_cnt,tf_fellow_cnt,tf_fellow_amt,tf_same_city_amt,tf_same_city_cnt,hb_log4_ttl_amt,hb_log4_ttl_cnt,hb_log4_max_amt,hb_log4_min_amt,hb_grp_amt,hb_grp_cnt,hb_grp_max_amt,hb_grp_min_amt,hb_grp_max_usr,hb_grp_min_usr,hb_low_5_cnt,hb_low_10_cnt,hb_low_15_cnt,hb_low_20_cnt,hb_high_20_cnt,hb_tgt_amt,hb_tgt_cnt,hb_tgt_max_amt,hb_tgt_min_amt,hb_tgt_usr_cnt,hb_rev_amt,hb_rev_cnt,hb_am_cnt,hb_pm_cnt,hb_em_cnt,hb_ng_cnt,hb_am_rev_cnt,hb_pm_rev_cnt,hb_em_rev_cnt,hb_ng_rev_cnt,hb_rev_max_amt,hb_rev_min_amt,hb_rev_usr_cnt,hb_grp_rev_amt,hb_grp_rev_cnt,hb_grp_rev_max_amt,hb_grp_rev_min_amt,hb_grp_rev_usr_cnt,hb_tgt_rev_amt,hb_tgt_rev_cnt,hb_tgt_rev_max_amt,hb_tgt_rev_min_amt,hb_tgt_rev_usr_cnt,tf_pm_rev_days,hb_pm_days,tf_am_rev_days,hb_am_days,hb_ng_days,ng_days,vir_am_days,tf_ng_rev_days,vir_pm_days,tf_em_rev_days,lct_pm_days,pm_days,lct_am_days,tf_pm_days,tf_em_days,hb_em_days,em_days,am_days,vir_em_days,vir_ng_days,tf_am_days,lct_ng_days,tf_ng_days,lct_em_days,fpay_days,hb_bpay_days,vir_bpay_days,tf_bpay_days,lct_fpay_days,tf_fpay_days,bpay_days,hb_whole_days,vir_fpay_days,tf_whole_days,hb_fpay_days,whole_days,vir_whole_days,lct_bpay_days,ttl_days,tf_rev_days,tf_rev_days_more1000,soc_bpay_ttl_amt,tf_ttl_more5000_days,tf_rev_ttl_more1000_days,tf_days_more5000,oth_ttl_more2000_days,tf_pm_rev_days_max,hb_pm_day

数据规模：(9098, 447)

标签比例：


0    4291
1    2831
Name: target_tov, dtype: int64

有空值的列:无
Index([], dtype='object')


In [3]:
#统计指标：
data1.describe()

,out_amount_nd,out_frequency_nd,out_amount_ss,out_frequency_ss,class,date_month,target_tov,acc_cnt,cre_type,ucreate_days,user_type,wx_firdate_days,wx_regtime_days,wxu_sex,if_bindmobile_safe_max,wxu_bindqq_safe,pay_crd_num,birth_date,sex,if_bindmobile_safe_min,if_chg_mob_max,if_chg_mob_min,if_chg_qq_max,if_chg_qq_min,if_chg_dev_max,if_chg_dev_min,chg_dev_cnt_max,chg_dev_cnt_min,if_op_safdev_max,if_op_safdev_min,unbind_max,unbind_min,old_acc_sus_max,old_acc_sus_min,maochi_lvl_max,maochi_lvl_min,dev_acc_cnt_max,dev_acc_cnt_min,acc_dev_cnt_max,acc_dev_cnt_min,ios_max,ios_min,block_max,block_min,trust_lvl_min,stol_lvl_max,feed_lvl_max,feed_lvl_min,anping_feed_max,anping_feed_min,sms_ttl_cnt,sms_cnt_max,sms_cnt_min,block_ttl_cnt,block_cnt_max,hb_ttl_amt,hb_ttl_cnt,hb_high_40_cnt,hb_low_3_cnt,hb_avg_low_1_cnt,hb_avg_high_100_cnt,hb_max_amt,hb_fpay_ttl_amt,hb_fpay_ttl_cnt,hb_bpay_ttl_amt,hb_bpay_ttl_cnt,hb_am_ttl_cnt,hb_pm_ttl_cnt,hb_ng_ttl_cnt,hb_em_ttl_cnt,tf_ttl_amt,tf_ttl_cnt,tf_max_amt,tf_min_amt,tf_usr_cnt,tf_am_ttl_cnt,tf_pm_ttl_cnt,tf_ng_ttl_cnt,tf_em_ttl_cnt,tf_rev_ttl_amt,tf_rev_ttl_cnt,tf_rev_max_amt,tf_rev_min_amt,tf_am_rev_ttl_cnt,tf_pm_rev_ttl_cnt,tf_ng_rev_ttl_cnt,tf_em_rev_ttl_cnt,tf_fpay_ttl_amt,tf_bpay_ttl_amt,tf_fpay_ttl_cnt,tf_bpay_ttl_cnt,lct_ttl_amt,lct_ttl_cnt,lct_max_amt,lct_prod_cnt,lct_fpay_ttl_amt,lct_fpay_ttl_cnt,lct_bpay_ttl_amt,lct_bpay_ttl_cnt,lct_am_ttl_cnt,lct_pm_ttl_cnt,lct_ng_ttl_cnt,lct_em_ttl_cnt,vir_ttl_amt,vir_ttl_cnt,vir_max_amt,vir_fpay_amt,vir_fpay_cnt,vir_bpay_amt,vir_bpay_cnt,vir_am_ttl_cnt,vir_pm_ttl_cnt,vir_ng_ttl_cnt,vir_em_ttl_cnt,traffic_ticket_cnt,phone_rate_cnt,qq_service_cnt,commonweal_cnt,commonweal_amt,safe_pay_amt,cre_crd_amt,cre_crd_cnt,self_wxcc_amt,other_wxcc_amt,self_wxcc_cnt,other_wxcc_cnt,ttl_amt,ttl_cnt,fpay_ttl_amt,fpay_ttl_cnt,bpay_ttl_amt,bpay_ttl_cnt,am_ttl_cnt,pm_ttl_cnt,ng_ttl_cnt,em_ttl_cnt,region_3_unify_cnt,tf_rev_usr_cnt,welfare_amt,welfare_cnt,welfare_max_amt,tencent_amt,tencent_cnt,tencent_max_amt,ecc_amt,ecc_cnt,ecc_max_amt,living_amt,living_cnt,living_max_amt,game_pay_amt,game_pay_cnt,game_pay_max_amt,game_pay_min_amt,web_vir_amt,web_vir_cnt,web_vir_max_amt,mob_rechg_amt,mob_rechg_cnt,mob_rechg_max_amt,trav_hotel_amt,trav_hotel_cnt,trav_hotel_max_amt,life_sev_amt,life_sev_cnt,life_sev_max_amt,mob_rechg_3th_amt,mob_rechg_3th_cnt,mob_rechg_3th_max_amt,mall_market_amt,mall_market_cnt,mall_market_max_amt,logis_amt,logis_cnt,logis_max_amt,beauty_amt,beauty_cnt,beauty_max_amt,car_amt,car_cnt,car_max_amt,baby_amt,baby_cnt,baby_max_amt,o2o_amt,o2o_cnt,o2o_max_amt,video_amt,video_cnt,video_max_amt,film_amt,film_cnt,film_max_amt,travel_amt,travel_cnt,travel_max_amt,tf_resid_cnt,tf_resid_amt,tf_card_cnt,tf_card_amt,tf_strange_amt,tf_strange_cnt,tf_fellow_cnt,tf_fellow_amt,tf_same_city_amt,tf_same_city_cnt,hb_log4_ttl_amt,hb_log4_ttl_cnt,hb_log4_max_amt,hb_log4_min_amt,hb_grp_amt,hb_grp_cnt,hb_grp_max_amt,hb_grp_min_amt,hb_grp_max_usr,hb_grp_min_usr,hb_low_5_cnt,hb_low_10_cnt,hb_low_15_cnt,hb_low_20_cnt,hb_high_20_cnt,hb_tgt_amt,hb_tgt_cnt,hb_tgt_max_amt,hb_tgt_min_amt,hb_tgt_usr_cnt,hb_rev_amt,hb_rev_cnt,hb_am_cnt,hb_pm_cnt,hb_em_cnt,hb_ng_cnt,hb_am_rev_cnt,hb_pm_rev_cnt,hb_em_rev_cnt,hb_ng_rev_cnt,hb_rev_max_amt,hb_rev_min_amt,hb_rev_usr_cnt,hb_grp_rev_amt,hb_grp_rev_cnt,hb_grp_rev_max_amt,hb_grp_rev_min_amt,hb_grp_rev_usr_cnt,hb_tgt_rev_amt,hb_tgt_rev_cnt,hb_tgt_rev_max_amt,hb_tgt_rev_min_amt,hb_tgt_rev_usr_cnt,tf_pm_rev_days,hb_pm_days,tf_am_rev_days,hb_am_days,hb_ng_days,ng_days,vir_am_days,tf_ng_rev_days,vir_pm_days,tf_em_rev_days,lct_pm_days,pm_days,lct_am_days,tf_pm_days,tf_em_days,hb_em_days,em_days,am_days,vir_em_days,vir_ng_days,tf_am_days,lct_ng_days,tf_ng_days,lct_em_days,fpay_days,hb_bpay_days,vir_bpay_days,tf_bpay_days,lct_fpay_days,tf_fpay_days,bpay_days,hb_whole_days,vir_fpay_days,tf_whole_days,hb_fpay_days,whole_days,vir_whole_days,lct_bpay_days,ttl_days,tf_rev_days,tf_rev_days_more1000,soc_bpay_ttl_amt,tf_ttl_more5000_days,tf_rev_ttl_more1000_days,tf_da

## 划分训练集、测试集

In [4]:
# TODO：将特征和标签分为两个数据集

x=data1[data1.columns.difference(['target_tov'])]
y=data1['target_tov']

# TODO：将数据集划分为训练集和测试集
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

## 未筛选特征进行分类lr，xgb


In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

lr = LogisticRegression()                                        # 实例化一个LR模型
lr.fit(x_train,y_train)                                          # 训练模型
y_prob = lr.predict_proba(x_test)[:,1]                           # 预测1类的概率
y_pred = lr.predict(x_test)                                      # 模型对测试集的预测结果

from sklearn import metrics
print ('AUC: %.4f' % metrics.roc_auc_score(y_test,y_prob))
print ('ACC: %.4f' % metrics.accuracy_score(y_test,y_pred))
print ('Recall: %.4f' % metrics.recall_score(y_test,y_pred))
print ('F1-score: %.4f' %metrics.f1_score(y_test,y_pred))
print ('Precesion: %.4f' %metrics.precision_score(y_test,y_pred))
metrics.confusion_matrix(y_test,y_pred)

AUC: 0.6176
ACC: 0.6154
Recall: 0.1715
F1-score: 0.2674
Precesion: 0.6061


g:\pycharmvenvs\jupyterwork\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


array([[777,  65],
       [483, 100]], dtype=int64)

In [6]:
from xgboost.sklearn import XGBClassifier
params = {
    'booster': 'gbtree',
    'objective': 'multi:softmax',  # 多分类的问题
    'num_class': 2,               # 类别数，与 multisoftmax 并用
    'gamma': 0.1,                  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
    'max_depth': 12,               # 构建树的深度，越大越容易过拟合
    'lambda': 2,                   # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
    'subsample': 0.7,              # 随机采样训练样本
    'colsample_bytree': 0.7,       # 生成树时进行的列采样
    'min_child_weight': 3,
    'silent': 1,                   # 设置成1则没有运行信息输出，最好是设置为0.
    'eta': 0.007,                  # 如同学习率
    'seed': 1000,
    'nthread': 4,                  # cpu 线程数
}
num_round = 10
import xgboost as xgb
dtrain = xgb.DMatrix(x_train, label=y_train)
dtest = xgb.DMatrix(x_test, label=y_test)
evallist = [ (dtrain, 'train')]
bst = xgb.train(params, dtrain, num_round, evallist)
y_prob = bst.predict(dtest)
y_pred = (y_prob >= 0.5)*1# AUC得分
                # 模型准确率

from sklearn import metrics
print ('AUC: %.4f' % metrics.roc_auc_score(y_test,y_prob))
print ('ACC: %.4f' % metrics.accuracy_score(y_test,y_pred))
print ('Recall: %.4f' % metrics.recall_score(y_test,y_pred))
print ('F1-score: %.4f' %metrics.f1_score(y_test,y_pred))
print ('Precesion: %.4f' %metrics.precision_score(y_test,y_pred))
metrics.confusion_matrix(y_test,y_pred)

[0]	train-merror:0.166403
[1]	train-merror:0.127962
[2]	train-merror:0.110233
[3]	train-merror:0.103388
[4]	train-merror:0.09426
[5]	train-merror:0.091452
[6]	train-merror:0.086712
[7]	train-merror:0.083377
[8]	train-merror:0.078638
[9]	train-merror:0.076005
AUC: 0.6354
ACC: 0.6702
Recall: 0.4443
F1-score: 0.5243
Precesion: 0.6395


array([[696, 146],
       [324, 259]], dtype=int64)

In [9]:
#pca降维

from sklearn.decomposition import PCA
pca=PCA(n_components=10)
X_pca_train=pca.fit_transform(x_train)
X_pca_test=pca.fit_transform(x_test)

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
lr = LogisticRegression() # 实例化一个LR模型
lr.fit(X_pca_train,y_train)                                          # 训练模型
y_pca_prob = lr.predict_proba(X_pca_test)[:,1]                           # 预测1类的概率
y_pca_pred = lr.predict(X_pca_test)                                      # 模型对测试集的预测结果

from sklearn import metrics
print ('AUC: %.4f' % metrics.roc_auc_score(y_test,y_pca_prob))
print ('ACC: %.4f' % metrics.accuracy_score(y_test,y_pca_pred))
print ('Recall: %.4f' % metrics.recall_score(y_test,y_pca_pred))
print ('F1-score: %.4f' %metrics.f1_score(y_test,y_pca_pred))
print ('Precesion: %.4f' %metrics.precision_score(y_test,y_pca_pred))
metrics.confusion_matrix(y_test,y_pca_pred)

AUC: 0.5116
ACC: 0.4891
Recall: 0.6021
F1-score: 0.4909
Precesion: 0.4144


array([[346, 496],
       [232, 351]], dtype=int64)

In [11]:
from xgboost.sklearn import XGBClassifier
params = {
    'booster': 'gbtree',
    'objective': 'multi:softmax',  # 多分类的问题
    'num_class': 2,               # 类别数，与 multisoftmax 并用
    'gamma': 0.1,                  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
    'max_depth': 12,               # 构建树的深度，越大越容易过拟合
    'lambda': 2,                   # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
    'subsample': 0.7,              # 随机采样训练样本
    'colsample_bytree': 0.7,       # 生成树时进行的列采样
    'min_child_weight': 3,
    'silent': 1,                   # 设置成1则没有运行信息输出，最好是设置为0.
    'eta': 0.007,                  # 如同学习率
    'seed': 1000,
    'nthread': 4,                  # cpu 线程数
}
num_round = 10
import xgboost as xgb
dtrain = xgb.DMatrix(X_pca_train, label=y_train)
dtest = xgb.DMatrix(X_pca_test, label=y_test)
evallist = [ (dtrain, 'train')]
bst = xgb.train(params, dtrain, num_round, evallist)
y_pca_prob = bst.predict(dtest)
y_pca_pred = (y_pca_prob >= 0.5)*1# AUC得分
                # 模型准确率

from sklearn import metrics
print ('AUC: %.4f' % metrics.roc_auc_score(y_test,y_pca_prob))
print ('ACC: %.4f' % metrics.accuracy_score(y_test,y_pca_pred))
print ('Recall: %.4f' % metrics.recall_score(y_test,y_pca_pred))
print ('F1-score: %.4f' %metrics.f1_score(y_test,y_pca_pred))
print ('Precesion: %.4f' %metrics.precision_score(y_test,y_pca_pred))
metrics.confusion_matrix(y_test,y_pca_pred)

[0]	train-merror:0.258908
[1]	train-merror:0.232403
[2]	train-merror:0.215903
[3]	train-merror:0.213797
[4]	train-merror:0.2066
[5]	train-merror:0.201861
[6]	train-merror:0.197648
[7]	train-merror:0.192031
[8]	train-merror:0.190451
[9]	train-merror:0.190627
AUC: 0.5415
ACC: 0.5860
Recall: 0.2967
F1-score: 0.3697
Precesion: 0.4901


array([[662, 180],
       [410, 173]], dtype=int64)

## 随机森林特征选择

In [12]:
## TODO：使用随机森林进行特征选择，首先在训练集拟合随机森林模型

rf = RandomForestClassifier(n_estimators=400,max_depth=10,verbose=1)
rf.fit(x_train,y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:    9.8s finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=400,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=1, warm_start=False)

In [13]:
imp=np.argsort(rf.feature_importances_)[::-1]
print("前230个特征的重要性贡献值：")
print(sum(rf.feature_importances_[imp[:230]]))
print('他们是：')
print(x_train.columns[list(np.argsort(rf.feature_importances_)[::-1])[:230]])
#前230个特征可以让贡献率和到达90%
X_RF_train=x_train[x_train.columns[list(np.argsort(rf.feature_importances_)[::-1])[:230]]]
X_RF_train.shape
X_RF_test=x_test[x_train.columns[list(np.argsort(rf.feature_importances_)[::-1])[:230]]]


前230个特征的重要性贡献值：
0.8951394214877716
他们是：
Index(['wx_regtime_days', 'ucreate_days', 'fpay_days', 'cre_crd_amt',
       'em_ttl_cnt', 'tf_em_ttl_cnt', 'wx_firdate_days', 'wxu_activity_score',
       'tf_fpay_ttl_amt', 'fpay_ttl_amt',
       ...
       'hb_fpay_days_max', 'o2o_cnt', 'tencent_cnt', 'maochi_lvl_min',
       'travel_days_max', 'tf_rev_days_more1000_max', 'tf_bpay_days_min',
       'am_days_min', 'vir_am_days', 'hb_fpay_days'],
      dtype='object', length=230)


In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
lr = LogisticRegression() # 实例化一个LR模型
lr.fit(X_RF_train,y_train)                                          # 训练模型
y_RF_prob = lr.predict_proba(X_RF_test)[:,1]                           # 预测1类的概率
y_RF_pred = lr.predict(X_RF_test)                                      # 模型对测试集的预测结果

from sklearn import metrics
print ('AUC: %.4f' % metrics.roc_auc_score(y_test,y_RF_prob))
print ('ACC: %.4f' % metrics.accuracy_score(y_test,y_RF_pred))
print ('Recall: %.4f' % metrics.recall_score(y_test,y_RF_pred))
print ('F1-score: %.4f' %metrics.f1_score(y_test,y_RF_pred))
print ('Precesion: %.4f' %metrics.precision_score(y_test,y_RF_pred))
metrics.confusion_matrix(y_test,y_RF_pred)

AUC: 0.6205
ACC: 0.5965
Recall: 0.0326
F1-score: 0.0620
Precesion: 0.6333


g:\pycharmvenvs\jupyterwork\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


array([[831,  11],
       [564,  19]], dtype=int64)

In [17]:
from xgboost.sklearn import XGBClassifier
params = {
    'booster': 'gbtree',
    'objective': 'multi:softmax',  # 多分类的问题
    'num_class': 2,               # 类别数，与 multisoftmax 并用
    'gamma': 0.1,                  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
    'max_depth': 12,               # 构建树的深度，越大越容易过拟合
    'lambda': 2,                   # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
    'subsample': 0.7,              # 随机采样训练样本
    'colsample_bytree': 0.7,       # 生成树时进行的列采样
    'min_child_weight': 3,
    'silent': 1,                   # 设置成1则没有运行信息输出，最好是设置为0.
    'eta': 0.007,                  # 如同学习率
    'seed': 1000,
    'nthread': 4,                  # cpu 线程数
}
num_round = 10
import xgboost as xgb
dtrain = xgb.DMatrix(X_RF_train, label=y_train)
dtest = xgb.DMatrix(X_RF_test, label=y_test)
evallist = [ (dtrain, 'train')]
bst = xgb.train(params, dtrain, num_round, evallist)
y_RF_prob = bst.predict(dtest)
y_RF_pred = (y_RF_prob >= 0.5)*1# AUC得分
                # 模型准确率

from sklearn import metrics
print ('AUC: %.4f' % metrics.roc_auc_score(y_test,y_RF_prob))
print ('ACC: %.4f' % metrics.accuracy_score(y_test,y_RF_pred))
print ('Recall: %.4f' % metrics.recall_score(y_test,y_RF_pred))
print ('F1-score: %.4f' %metrics.f1_score(y_test,y_RF_pred))
print ('Precesion: %.4f' %metrics.precision_score(y_test,y_RF_pred))
metrics.confusion_matrix(y_test,y_RF_pred)

[0]	train-merror:0.174653
[1]	train-merror:0.128313
[2]	train-merror:0.111287
[3]	train-merror:0.103388
[4]	train-merror:0.098122
[5]	train-merror:0.092856
[6]	train-merror:0.091803
[7]	train-merror:0.087239
[8]	train-merror:0.083026
[9]	train-merror:0.080393
AUC: 0.6321
ACC: 0.6660
Recall: 0.4460
F1-score: 0.5221
Precesion: 0.6295


array([[689, 153],
       [323, 260]], dtype=int64)

## 当前来看，随机森林特征选择，加xgb效果最好，进一步加入归一化

In [16]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
# ss = StandardScaler()
ss = MinMaxScaler()
X_RF_st_train = ss.fit_transform(X_RF_train)
X_RF_st_test = ss.fit_transform(X_RF_test)
from xgboost.sklearn import XGBClassifier
params = {
    'booster': 'gbtree',
    'objective': 'multi:softmax',  # 多分类的问题
    'num_class': 2,               # 类别数，与 multisoftmax 并用
    'gamma': 0.1,                  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
    'max_depth': 12,               # 构建树的深度，越大越容易过拟合
    'lambda': 2,                   # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
    'subsample': 0.7,              # 随机采样训练样本
    'colsample_bytree': 0.7,       # 生成树时进行的列采样
    'min_child_weight': 3,
    'silent': 1,                   # 设置成1则没有运行信息输出，最好是设置为0.
    'eta': 0.007,                  # 如同学习率
    'seed': 1000,
    'nthread': 4,                  # cpu 线程数
}
num_round = 10
import xgboost as xgb
dtrain = xgb.DMatrix(X_RF_st_train, label=y_train)
dtest = xgb.DMatrix(X_RF_st_test, label=y_test)
evallist = [ (dtrain, 'train')]
bst = xgb.train(params, dtrain, num_round, evallist)
y_RF_prob = bst.predict(dtest)
y_RF_pred = (y_RF_prob >= 0.5)*1# AUC得分
                # 模型准确率

from sklearn import metrics
print ('AUC: %.4f' % metrics.roc_auc_score(y_test,y_RF_prob))
print ('ACC: %.4f' % metrics.accuracy_score(y_test,y_RF_pred))
print ('Recall: %.4f' % metrics.recall_score(y_test,y_RF_pred))
print ('F1-score: %.4f' %metrics.f1_score(y_test,y_RF_pred))
print ('Precesion: %.4f' %metrics.precision_score(y_test,y_RF_pred))
metrics.confusion_matrix(y_test,y_RF_pred)

[0]	train-merror:0.17518
[1]	train-merror:0.12884
[2]	train-merror:0.111111
[3]	train-merror:0.103212
[4]	train-merror:0.098473
[5]	train-merror:0.092856
[6]	train-merror:0.093734
[7]	train-merror:0.088994
[8]	train-merror:0.085835
[9]	train-merror:0.079516
AUC: 0.6335
ACC: 0.6526
Recall: 0.5283
F1-score: 0.5545
Precesion: 0.5833


array([[622, 220],
       [275, 308]], dtype=int64)